In [15]:
!pip install cryptography --quiet
print('cryptography installed')

cryptography installed


In [16]:
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM, ChaCha20Poly1305

# Helper: write sample file
sample_text = 'This is a secret backup message\n'* 3
with open('sample.txt','w') as f:
    f.write(sample_text)
print('sample.txt created')

sample.txt created


In [17]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import os

# --- Key Generation (AES-256) ---
def generate_aes_key():
    return AESGCM.generate_key(bit_length=256)

# --- Encrypt file with AES-GCM (nonce + ciphertext stored) ---
def encrypt_file_aes(filepath, key):
    aesgcm = AESGCM(key)
    nonce = os.urandom(12)  # 96-bit
    with open(filepath,'rb') as f:
        pt = f.read()
    ct = aesgcm.encrypt(nonce, pt, None)
    out = filepath + '.enc'
    with open(out,'wb') as f:
        f.write(nonce + ct)
    return out

# --- Decrypt file ---
def decrypt_file_aes(enc_filepath, key):
    with open(enc_filepath,'rb') as f:
        data = f.read()
    nonce = data[:12]
    ct = data[12:]
    aesgcm = AESGCM(key)
    pt = aesgcm.decrypt(nonce, ct, None)
    return pt

print('AES-GCM functions defined')

AES-GCM functions defined


In [18]:
key = generate_aes_key()
enc = encrypt_file_aes('sample.txt', key)
print('Encrypted file:', enc)
pt = decrypt_file_aes(enc, key)
print('Decrypted preview:\n', pt.decode()[:200])

Encrypted file: sample.txt.enc
Decrypted preview:
 This is a secret backup message
This is a secret backup message
This is a secret backup message



In [19]:
print('Testing tamper detection...')
# Tamper with ciphertext
with open(enc, 'rb') as f:
    data = bytearray(f.read())
# flip a byte in ciphertext (after nonce)
data[20] ^= 1
with open('tampered.enc','wb') as f:
    f.write(data)

try:
    _ = decrypt_file_aes('tampered.enc', key)
    print('Tampered file decrypted (unexpected)')
except Exception as e:
    print('Tamper detected, decryption failed as expected:', type(e).__name__)

Testing tamper detection...
Tamper detected, decryption failed as expected: InvalidTag


## Optional: ChaCha20-Poly1305 variant
Use this if target devices may lack AES-NI hardware.


In [20]:
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305

# Key generation
def generate_chacha_key():
    return ChaCha20Poly1305.generate_key()

# Encrypt/decrypt

def encrypt_file_chacha(filepath, key):
    a = ChaCha20Poly1305(key)
    nonce = os.urandom(12)
    with open(filepath,'rb') as f:
        pt = f.read()
    ct = a.encrypt(nonce, pt, None)
    out = filepath + '.chacha.enc'
    with open(out,'wb') as f:
        f.write(nonce + ct)
    return out


def decrypt_file_chacha(enc_filepath, key):
    with open(enc_filepath,'rb') as f:
        data = f.read()
    nonce = data[:12]
    ct = data[12:]
    a = ChaCha20Poly1305(key)
    pt = a.decrypt(nonce, ct, None)
    return pt

print('ChaCha20 functions ready')

ChaCha20 functions ready


In [21]:
ch_key = generate_chacha_key()
ch_enc = encrypt_file_chacha('sample.txt', ch_key)
print('ChaCha encrypted:', ch_enc)
print(decrypt_file_chacha(ch_enc, ch_key)[:200])

ChaCha encrypted: sample.txt.chacha.enc
b'This is a secret backup message\nThis is a secret backup message\nThis is a secret backup message\n'
